## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The exampe uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server. 

In [9]:
import mlflow
import os

os.environ["AWS_PROFILE"] = "uvezero-profile" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "ec2-13-48-44-238.eu-north-1.compute.amazonaws.com" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [10]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-13-48-44-238.eu-north-1.compute.amazonaws.com:5000'


In [11]:
mlflow.search_experiments() # list_experiments API has been removed, you can use search_experiments instead.()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-juan/1', creation_time=1724065552853, experiment_id='1', last_update_time=1724065552853, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-juan/0', creation_time=1724063719490, experiment_id='0', last_update_time=1724063719490, lifecycle_stage='active', name='Default', tags={}>]

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

/home/juan/miniconda3/envs/mlops_venv/lib/python3.9/site-packages/_distutils_hack/__init__.py:17: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/juan/miniconda3/envs/mlops_venv/lib/python3.9/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


default artifacts URI: 's3://mlflow-artifacts-remote-juan/1/315bfedf23ce48529866d618f9b9d240/artifacts'


In [13]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-juan/1', creation_time=1724065552853, experiment_id='1', last_update_time=1724065552853, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-juan/0', creation_time=1724063719490, experiment_id='0', last_update_time=1724063719490, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [14]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [16]:
client.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-juan/1', creation_time=1724065552853, experiment_id='1', last_update_time=1724065552853, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-juan/0', creation_time=1724063719490, experiment_id='0', last_update_time=1724063719490, lifecycle_stage='active', name='Default', tags={}>]

In [20]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Registered model 'iris-classifier' already exists. Creating a new version of this model...
2024/08/19 13:58:08 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1724068688062, current_stage='None', description='', last_updated_timestamp=1724068688062, name='iris-classifier', run_id='315bfedf23ce48529866d618f9b9d240', run_link='', source='s3://mlflow-artifacts-remote-juan/1/315bfedf23ce48529866d618f9b9d240/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>